## Деревья решений 
### Ноутбук составлен по мотивам лекций профессора Воронцова по ML для МФТИ и ШАД

Чтобы поупражняться, воспользуемся датасетом Фишера с ирисами, взятым отсюда:
https://archive.ics.uci.edu/ml/datasets/iris

In [9]:
import pandas as pd
df_iris = pd.read_csv("iris.data")
df_iris.columns = ["sepal length", "sepal width", "petal length", "petal width", "class"]
df_iris.sample(5)

,sepal length,sepal width,petal length,petal width,class
112,5.7,2.5,5.0,2.0,Iris-virginica
23,4.8,3.4,1.9,0.2,Iris-setosa
108,7.2,3.6,6.1,2.5,Iris-virginica
47,5.3,3.7,1.5,0.2,Iris-setosa
121,7.7,2.8,6.7,2.0,Iris-virginica
